# Estimate Tests

- Create the internal consistency code
- build the diff file code for estimates 

In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import copy
import pandas as pd
import pyodbc
import os
import glob
import numpy as np

What internal consistency will do 


Functions: 
1. Function where you pass in the three levels and it checks for equivalence between them and outputs differences, maybe a helpful dataframe 
2. Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 
- Could output a dataframe where all the sums are there then filter for greater than zero, or output the whole dataframe 
3. Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 
4. Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 
5. Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [2]:
def internal_consistency():
    '''
    Along with an individual check for each column sum 

    Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 

Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 
    
    '''

# Internal Consistency

In [6]:

# importing the required modules
import glob
import pandas as pd
  
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

data = {}
for path in files:
    data[path.split('\\')[1][:-4]] = pd.read_csv(path)

In [7]:
data.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

### Compare Totals 
This is comparing the totals accross CPA, Jur, and Region to see if they are equivalent. 

In [8]:
def compare_totals(cpa_df, jur_df, region_df):
    """
    Aggregates sum of values by year in cpa, jurisdiction, and region level and compares them together to see if they match. CPA is removed as many areas calculated in the other georgraphy 
    """
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df
    
    non_matches_dict = {}
    jur_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(jur_totals[col], cpa_totals[col]).all():
            jur_non_matches.append(col)
                        
    if len(jur_non_matches) == len(cpa_totals.columns):
        non_matches_dict['jurisdiction'] = 'all columns did not match.'
    elif len(jur_non_matches) == 0:
        non_matches_dict['jurisdiction'] = 'all columns matched.'
    else:
        if len(jur_non_matches) > 10:
            non_matches_dict['jurisdiction'] = f"{len(jur_non_matches)} columns did not match out of {len(cpa_totals.columns)} columns."
        else:
            non_matches_dict['jurisdiction'] = ', '.join(jur_non_matches) + ' columns did not match.'
    
    reg_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(reg_totals[col], cpa_totals[col]).all():
            reg_non_matches.append(col)
    if len(reg_non_matches) == len(cpa_totals.columns):
        non_matches_dict['region'] = 'all columns did not match.'
    elif len(reg_non_matches) == 0:
        non_matches_dict['region'] = 'all columns matched.'
    else:
        if len(reg_non_matches) > 10:
            non_matches_dict['region'] = f"{len(reg_non_matches)} columns did not match out of {len(cpa_totals.columns)}."
        else:
            non_matches_dict['region'] = ', '.join(reg_non_matches) + ' columns did not match.'
    
    return non_matches_dict

In [9]:
compare_totals(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'jurisdiction': 'all columns matched.', 'region': 'all columns matched.'}

### Ethnicity population check 
Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

In [10]:
def ethnicity_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Hispanic'] + df['Non-Hispanic, White'] + df['Non-Hispanic, Black'] + df['Non-Hispanic, American Indian or Alaska Native'] + df['Non-Hispanic, Asian'] + df['Non-Hispanic, Hawaiian or Pacific Islander'] + df['Non-Hispanic, Other'] + df['Non-Hispanic, Two or More Races']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results


In [11]:
ethnicity_internal_consistency(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Age Group Comparison Check
Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

In [12]:
def age_group_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Under 5'] + df['5 to 9'] + df['10 to 14'] + df['15 to 17'] + df['18 and 19'] + df['20 to 24'] + df['25 to 29'] + df['30 to 34'] + df['35 to 39'] + df['40 to 44'] + df['45 to 49'] + df['50 to 54'] + df['55 to 59'] + df['60 and 61'] + df['62 to 64'] + df['65 to 69'] + df['70 to 74'] + df['75 to 79'] + df['80 to 84'] + df['85 and Older']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [13]:
age_group_internal_consistency(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Gender Population test 
Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [14]:
def gender_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Female'] + df['Male']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [15]:
gender_internal_consistency(data['2020_06_consolidated_cpa_QA'], data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

# Creating Diff Files

In [16]:
def diff_file(df1, df2, geo_level=None):
    '''
    This function creates a diff file of two dataframes with identical labels and identical shapes and creates a diff file of the two. 
    This does df1 minus df2
    
    '''
    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id']) - df2.set_index(['cpa', 'yr_id'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id']) - df2.set_index(['jurisdiction', 'yr_id'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id']) - df2.set_index(['region', 'yr_id'])).reset_index()

    
    return new_df

In [17]:
diff_file(data['2020_06_consolidated_jurisdiction_QA'], data['2020_06_consolidated_jurisdiction_QA'], geo_level='jurisdiction')

,jurisdiction,yr_id,Under 5,5 to 9,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,...,Mobile Home,Single-family Detached,Single-family Attached,Total Population,Household Population,Group Quarters - Military,Group Quarters - College,Group Quarters - Other,Female,Male
0,Carlsbad,2010,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Carlsbad,2011,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Carlsbad,2012,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Carlsbad,2013,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Carlsbad,2014,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Vista,2016,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,Vista,2017,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206,Vista,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207,Vista,2019,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Year on Year output file
- Is this meant to be for every file that we have? Or just the consolidated ones

In [67]:
def yoy_pct_diff(df, geo_level=None):
    '''
    This finds the percent change by year
    df must be already indexed
    
    '''
    if geo_level == None:
        print('Please input a valid geography level')
    elif geo_level == 'region':
        final = df.pct_change()*100
    elif geo_level == 'cpa':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    elif geo_level == 'jurisdiction':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    return final

In [80]:
yoy_pct_diff(data['2020_06_consolidated_jurisdiction_QA'].set_index(['jurisdiction', 'yr_id']), geo_level='jurisdiction')

Under 5    5 to 9  10 to 14  15 to 17  18 and 19  \
jurisdiction yr_id                                                      
Chula Vista  2010        NaN       NaN       NaN       NaN        NaN   
             2011   5.337159  2.640409 -1.246782 -1.254765  -5.302241   
             2012   4.200490  3.869446 -1.150072  1.359176   4.030816   
             2013   3.116177  2.069037  0.729097  3.435690   1.824914   
             2014   2.413548  2.209079  0.328542 -1.304081   1.675325   
...                      ...       ...       ...       ...        ...   
Carlsbad     2016   1.480315  0.654490  0.201845 -0.625539  -1.336222   
             2017  -0.294848 -0.680478  0.071942 -0.629477  -2.928258   
             2018   0.762646  2.024970 -1.135873  0.327654   0.188537   
             2019  -1.220266  0.746157  0.000000 -2.721533   1.618367   
             2020  -2.580141  2.932899  0.581734 -0.492390  -1.185185   

                    20 to 24  25 to 29  30 to 34  35 to 39  40 to 44  ...  \
jurisdiction yr_id                                                    ...   
Chula Vista  2010        NaN       NaN       NaN       NaN       NaN  ...   
             2011   9.146914  2.454062 -0.267475 -1.952026 -0.540161  ...   
             2012   6.595710  0.672566  1.978664 -2.755751 -1.064465  ...   
             2013   4.114977  1.025551  1.694816 -0.960037 -0.790470  ...   
             2014   2.324906  1.659029  1.603356 -1.191241 -0.547778  ...   
...                      ...       ...       ...       ...       ...  ...   
Carlsbad     2016  -1.202125  1.817597 -1.384638  2.330695 -5.796540  ...   
             2017   2.758913  1.943128 -2.822938  2.492480 -4.303138  ...   
             2018   0.275368 -0.650860  0.243346  0.600978 -1.045396  ...   
             2019  -2.855966 -2.261738 -0.728266  0.180606  0.101302  ...   
             2020   0.890459 -0.829876 -0.366804  0.554708  0.665028  ...   

                    Mobile Home  Single-family Detached  \
jurisdiction yr_id                                        
Chula Vista  2010           NaN                     NaN   
             2011      0.731776                     NaN   
             2012      0.698519                     NaN   
             2013      0.249723                     NaN   
             2014      0.000000                     NaN   
...                         ...                     ...   
Carlsbad     2016      0.000000                     NaN   
             2017      0.000000                     NaN   
             2018      0.251889                     NaN   
             2019      0.083752                     NaN   
             2020      0.000000                     NaN   

                    Single-family Attached  Total Population  \
jurisdiction yr_id                                             
Chula Vista  2010                      NaN               NaN   
             2011                      NaN          1.177865   
             2012                      NaN          1.682004   
             2013                      NaN          1.999283   
             2014                      NaN          1.433444   
...                                    ...               ...   
Carlsbad     2016                      NaN          0.589430   
             2017                      NaN          0.469492   
             2018                      NaN          0.570847   
             2019                      NaN          0.000000   
             2020                      NaN          0.728649   

                    Household Population  Group Quarters - Military  \
jurisdiction yr_id                                                    
Chula Vista  2010                    NaN                        NaN   
             2011               1.251136                        NaN   
             2012               1.650830                        NaN   
             2013               2.021199                        NaN   
             2014               1.420785  

# Proportion Check 
- Compare the proportion of groups (age/gender/ethnicity/ethnicity by gender by age in total population between DOF and Estimates at Region Level  

- To be performed on Age by Category, Gender, Ethnicity, Ethnicity by age and gender 


- What is category?
- Am I doing all of the proportion checks?


In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=socioec_data;'
                    'Trusted_Connection=yes;')

In [3]:
age_query = '''SELECT * 
FROM [ddamwsql16].[socioec_data].[ca_dof].[population_proj_2021_07_14]'''

age_df =  pd.read_sql_query(age_query, conn) #example df

### Age Proportion Check

In [81]:
data['2020_06_consolidated_region_QA']

,region,yr_id,Under 5,5 to 9,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,...,Mobile Home,Single-family Detached,Single-family Attached,Total Population,Household Population,Group Quarters - Military,Group Quarters - College,Group Quarters - Other,Female,Male
0,San Diego,2010,203423,194029,198716,128000,97095,270750,250737,220185,...,39846,0,0,3095314,2993348,43597,16055,42314,1541635,1553679
1,San Diego,2011,209818,197773,198866,128833,95984,269851,246589,224827,...,39956,0,0,3125655,3023755,42309,16968,42623,1555437,1570218
2,San Diego,2012,215677,203754,198403,131596,97211,267545,240615,230265,...,40058,0,0,3161809,3058317,42539,17540,43413,1572513,1589296
3,San Diego,2013,219753,210910,199295,133282,98736,265080,234627,235654,...,40119,0,0,3199901,3098470,40622,18316,42493,1590776,1609125
4,San Diego,2014,222064,216798,200875,132669,100101,263770,228231,240016,...,40168,0,0,3232763,3127530,42313,19046,43874,1606047,1626716
5,San Diego,2015,223824,221252,202795,131361,101520,263884,222382,240792,...,40169,0,0,3265039,3156961,43732,19357,44989,1620812,1644227
6,San Diego,2016,223355,224985,205244,131064,100003,263975,218370,236207,...,40222,0,0,3285151,3177681,44193,19160,44117,1630016,1655135
7,San Diego,2017,221684,228181,210062,131314,98316,264924,215777,230157,...,40081,0,0,3306890,3200591,43280,18989,44030,1640232,1666658
8,San Diego,2018,219100,229534,215718,131683,98570,264797,212733,223717,...,40020,0,0,3326318,3215545,43995,20178,46600,1649257,1677061
9,San Diego,2019,213921,230600,220580,132502,99494,263897,210274,216586,...,40116,0,0,3340312,3230523,44257,20208,45324,1655636,1684676


In [ ]:
'''SELECT dof.fiscal_yr AS 'Year', dof.sex, dim.ethnicity_id
FROM [ddamwsql16].[socioec_data].[ca_dof].[population_proj_2021_07_14] AS dof
LEFT JOIN
	(SELECT [ethnicity_id], [short_name]
	FROM [estimates].[est_2020_06].[dw_age]) AS dim
	ON (dof.[race_code] = dim.[ethnicity_id])'''